In [25]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import requests
from pandasql import sqldf
import json

In [26]:
print("Which Contest would you like to view?")
contest = input()

Which Contest would you like to view?


In [27]:
page = "https://codeforces.com/api/contest.status?contestId=" + contest + "&from=1"
print(page)

https://codeforces.com/api/contest.status?contestId=1676&from=1


In [28]:
cf_submissions_api = requests.get(page)

In [29]:
submissions = cf_submissions_api.json()

In [30]:
df = pd.json_normalize(submissions,['result'])

In [31]:
df.head(10)

,id,contestId,creationTimeSeconds,relativeTimeSeconds,programmingLanguage,verdict,testset,passedTestCount,timeConsumedMillis,memoryConsumedBytes,...,problem.type,problem.rating,problem.tags,author.contestId,author.members,author.participantType,author.ghost,author.startTimeSeconds,author.teamId,author.teamName
0,157371768,1676,1652689198,2147483647,GNU C++20 (64),OK,TESTS,6,15,1228800,...,PROGRAMMING,800,"[brute force, greedy, implementation, implemen...",1676,[{'handle': 'Kris_Wu'}],PRACTICE,False,1652193900,NaN,NaN
1,157371767,1676,1652689198,2147483647,GNU C++17,WRONG_ANSWER,TESTS,1,31,1024000,...,PROGRAMMING,1000,"[brute force, greedy, implementation]",1676,[{'handle': 'Nastya2109'}],PRACTICE,False,1652193900,NaN,NaN
2,157371724,1676,1652689158,1758,GNU C++17,OK,TESTS,31,155,8601600,...,PROGRAMMING,1100,"[binary search, greedy, sortings]",1676,[{'handle': 'dsyz'}],VIRTUAL,False,1652687400,NaN,NaN
3,157371716,1676,1652689151,2147483647,GNU C++17,OK,TESTS,6,15,1024000,...,PROGRAMMING,800,"[brute force, greedy, implementation, implemen...",1676,[{'handle': 'EmmaWatson'}],PRACTICE,False,1652193900,NaN,NaN
4,157371684,1676,1652689116,2147483647,GNU C++14,WRONG_ANSWER,TESTS,0,0,1024000,...,PROGRAMMING,800,[implementation],1676,[{'handle': 'anjalish'}],PRACTICE,False,1652193900,NaN,NaN
5,157371643,1676,1652689060,2147483647,GNU C++17,OK,TESTS,6,30,1024000,...,PROGRAMMING,800,"[brute force, greedy, implementation, implemen...",1676,[{'handle': 'co_m_der15'}],PRACTICE,False,1652193900,NaN,NaN
6,157371629,1676,1652689032,2147483647,GNU C++14,WRONG_ANSWER,TESTS,0,0,1024000,...,PROGRAMMING,800,[implementation],1676,[{'handle': 'anjalish'}],PRACTICE,False,1652193900,NaN,NaN
7,157371545,1676,1652688958,2147483647,GNU C++14,TIME_LIMIT_EXCEEDED,TESTS,11,1000,21708800,...,PROGRAMMING,1300,"[dfs and similar, dp, graphs, trees]",1676,[{'handle': 'tachyon_11'}],PRACTICE,False,1652193900,NaN,NaN
8,157371524,1676,1652688939,2147483647,GNU C++14,WRONG_ANSWER,TESTS,0,0,1024000,...,PROGRAMMING,800,[implementation],1676,[{'handle': 'anjalish'}],PRACTICE,False,1652193900,NaN,NaN
9,157371502,1676,1652688917,2147483647,GNU C++20 (64),TIME_LIMIT_EXCEEDED,TESTS,4,3500,1331200,...,PROGRAMMING,1100,"[binary search, greedy, sortings]",1676,[{'handle': 'XIExx'}],PRACTICE,False,1652193900,NaN,NaN


In [32]:
byTimeMemory = df[["relativeTimeSeconds","contestId","problem.index","programmingLanguage","verdict","timeConsumedMillis", "memoryConsumedBytes"]]

In [33]:
#to facilitate the sql queries
byTimeMemory = byTimeMemory.rename(columns = {"timeConsumedMillis": "Time","problem.index":"problem_index"})

This part finds the average runtime of accepted submissions per question per language

In [34]:
runtime = sqldf("select contestId, problem_index, programmingLanguage, Time, memoryConsumedBytes from byTimeMemory where verdict = 'OK' ")

In [35]:
runtime.head(10)

,contestId,problem_index,programmingLanguage,Time,memoryConsumedBytes
0,1676,C,GNU C++20 (64),15,1228800
1,1676,E,GNU C++17,155,8601600
2,1676,C,GNU C++17,15,1024000
3,1676,C,GNU C++17,30,1024000
4,1676,B,GNU C++14,78,1228800
5,1676,B,GNU C++20 (64),15,1228800
6,1676,E,GNU C++17,935,1638400
7,1676,F,GNU C++20 (64),156,13107200
8,1676,C,Python 3,685,4505600
9,1676,H1,GNU C++20 (64),15,2867200


In [36]:
runtime["Memory(KB)"] = runtime["memoryConsumedBytes"] // 1024
runtime = runtime.drop(columns = ["memoryConsumedBytes"])
runtime

,contestId,problem_index,programmingLanguage,Time,Memory(KB)
0,1676,C,GNU C++20 (64),15,1200
1,1676,E,GNU C++17,155,8400
2,1676,C,GNU C++17,15,1000
3,1676,C,GNU C++17,30,1000
4,1676,B,GNU C++14,78,1200
...,...,...,...,...,...
118653,1676,A,GNU C++20 (64),15,0
118654,1676,A,GNU C++20 (64),15,1600
118655,1676,A,GNU C++20 (64),15,0
118656,1676,A,GNU C++14,15,0


In [50]:
#example
runtime.loc[(runtime["problem_index"] == "G") & (runtime["programmingLanguage"] == "PyPy 3-64")]

,contestId,problem_index,programmingLanguage,Time,Memory(KB)
737,1676,G,PyPy 3-64,919,251800
5254,1676,G,PyPy 3-64,483,20900
6863,1676,G,PyPy 3-64,810,29900
7397,1676,G,PyPy 3-64,514,12700
9137,1676,G,PyPy 3-64,997,32700
...,...,...,...,...,...
70113,1676,G,PyPy 3-64,623,21300
70469,1676,G,PyPy 3-64,701,145200
72667,1676,G,PyPy 3-64,888,214600
75427,1676,G,PyPy 3-64,998,257100


In [37]:
runtime_groupby = runtime.groupby(by = ["contestId", "problem_index", "programmingLanguage"]).mean().round(1)

In [38]:
runtime_groupby

Time  Memory(KB)
contestId problem_index programmingLanguage                     
1676      A             C# 10                   55.9      1709.1
                        C# 8                    82.5       515.4
                        Clang++17 Diagnostics   32.3       585.7
                        D                       15.0         0.0
                        Delphi                  15.0       100.0
...                                              ...         ...
          H2            MS C++ 2017            229.3     13793.3
                        Node.js                337.3     30566.7
                        PyPy 3                 387.2     20429.0
                        PyPy 3-64              407.0     31766.0
                        Rust 2021               33.6      7740.0

[253 rows x 2 columns]

In [39]:
runtime_groupby.to_csv("G:/" + contest + "_runtime_stats.csv")